In [1]:
import sys
path = '/home/mkarri/extpersonalization/src'
if path not in sys.path:
    sys.path.append(path)
from models.sequential.GRU4Rec import GRU4Rec
from torch.utils.data import DataLoader
from helpers import BaseRunner
import torch
import torch.nn as nn
import numpy as np
import pickle
from time import time
import gc
from tqdm import tqdm
from utils import utils


In [2]:
with open('../data/ml-1m/args.pkl','rb') as f:
    args = pickle.load(f)
with open('../data/ml-1m/SeqReader.pkl','rb') as f:
    corpus = pickle.load(f)
    model = GRU4Rec(args,corpus)
file_path = '../model/GRU4Rec/GRU4Rec__ml-1m__0__lr=0.001__l2=0__emb_size=64__hidden_size=64.pt'
model.load_state_dict(torch.load(file_path))

<All keys matched successfully>

In [3]:
class GRU4Recold(GRU4Rec):
    reader = 'SeqReader'
    runner = 'BaseRunner'
    extra_log_args = ['emb_size', 'hidden_size']

    @staticmethod
    def parse_model_args(parser):
        parser.add_argument('--emb_size', type=int, default=64,
                            help='Size of embedding vectors.')
        parser.add_argument('--hidden_size', type=int, default=64,
                            help='Size of hidden vectors in GRU.')
        return SequentialModel.parse_model_args(parser)

    def __init__(self, args, corpus):
        super().__init__(args, corpus)
        self.emb_size = args.emb_size
        self.hidden_size = args.hidden_size
        self._define_params()
        self.apply(self.init_weights)

    def _define_params(self):
        self.i_embeddings = nn.Embedding(self.item_num, self.emb_size)
        self.rnn = nn.GRU(input_size=self.emb_size, hidden_size=self.hidden_size, batch_first=True)
        # self.pred_embeddings = nn.Embedding(self.item_num, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.emb_size)
    def loss(self, out_dict: dict) -> torch.Tensor:
        """
        BPR ranking loss with optimization on multiple negative samples (a little different now)
        "Recurrent neural networks with top-k gains for session-based recommendations"
        :param out_dict: contain prediction with [batch_size, -1], the first column for positive, the rest for negative
        :return:
        """
        predictions = out_dict['prediction']
        pos_pred, neg_pred = predictions[:, 0], predictions[:, 1:]
        neg_softmax = (neg_pred - neg_pred.max()).softmax(dim=1)
        loss = -((pos_pred[:, None] - neg_pred).sigmoid() * neg_softmax).sum(dim=1).log().mean()
        # neg_pred = (neg_pred * neg_softmax).sum(dim=1)
        # loss = F.softplus(-(pos_pred - neg_pred)).mean()
        # ↑ For numerical stability, use 'softplus(-x)' instead of '-log_sigmoid(x)'
        return loss

    def forward(self, feed_dict):
        #self.check_list = []
        i_ids = feed_dict['item_id']  # [batch_size, -1]
        history = feed_dict['history_items']  # [batch_size, history_max]
        lengths = feed_dict['lengths']  # [batch_size]

        his_vectors = self.i_embeddings(history)

        # Sort and Pack
        sort_his_lengths, sort_idx = torch.topk(lengths, k=len(lengths))
        sort_his_vectors = his_vectors.index_select(dim=0, index=sort_idx)
        history_packed = torch.nn.utils.rnn.pack_padded_sequence(
            sort_his_vectors, sort_his_lengths.cpu(), batch_first=True)

        # RNN
        output, hidden = self.rnn(history_packed, None)

        # Unsort
        unsort_idx = torch.topk(sort_idx, k=len(lengths), largest=False)[1]
        rnn_vector = hidden[-1].index_select(dim=0, index=unsort_idx)

        # Predicts
        # pred_vectors = self.pred_embeddings(i_ids)
        pred_vectors = self.i_embeddings(i_ids)
        rnn_vector = self.out(rnn_vector)
        prediction = (rnn_vector[:, None, :] * pred_vectors).sum(-1)
        return {'prediction': prediction.view(feed_dict['batch_size'], -1)}
    
    def predict_interaction(self,feed_dict,topk = 10):
        pred = self.forward(feed_dict)['prediction']
        sort_idx = (-pred).argsort(axis=1)
        return sort_idx[:topk]
        

In [4]:
model_check = GRU4Recold(args,corpus)
model_check.load_state_dict(torch.load(file_path))

<All keys matched successfully>

In [5]:
## list of all items present
item_id = [list(range(1,3707))]
## interaction history of items
history_items = [[907, 2915,  335, 1248,  514,  853, 3204,  312, 1157, 1161, 1486, 2522,
         528, 2006, 1169, 1176, 1193, 3108,  882, 1092]]
## user id
user_id = [6034]
## lengths
lengths = [len(x) for x in history_items]
batch_size = len(user_id)
sample = {'user_id':torch.tensor(user_id),'item_id':torch.tensor(item_id),'history_items':torch.tensor(history_items),\
         'lengths':torch.tensor(lengths),'batch_size':batch_size}

In [6]:
model_check.predict_interaction(sample)

tensor([[ 158, 3107, 1091,  ..., 2950, 1991,  774]])

In [ ]:
data_dict = dict()
for phase in ['train', 'dev', 'test']:
    data_dict[phase] = GRU4Rec.Dataset(model_check, corpus, phase)
    data_dict[phase].prepare()

### Get the BaseRunner model

In [ ]:
class BaseRunnerv1(BaseRunner.BaseRunner):
    def train(self, data_dict):
        model = data_dict['train'].model
        main_metric_results, dev_results = list(), list()
        self._check_time(start=True)
        try:
            for epoch in range(self.epoch):
                # Fit
                self._check_time()
                gc.collect()
                torch.cuda.empty_cache()
                loss = self.fit(data_dict['train'], epoch=epoch + 1)
                training_time = self._check_time()
                train_params = {'training_time':training_time,'train_loss':loss}
                self.mlflow_log_metrics(train_params,epoch)

                # Observe selected tensors
                if len(model.check_list) > 0 and self.check_epoch > 0 and epoch % self.check_epoch == 0:
                    utils.check(model.check_list)

                # Record dev results
                dev_result = self.evaluate(data_dict['dev'], self.topk, self.metrics)
                self.mlflow_log_metrics(self.head_to_dict(dev_result,'dev'),epoch)
                dev_results.append(dev_result)
                
                main_metric_results.append(dev_result[self.main_metric])
                logging_str = 'Epoch {:<5} loss={:<.4f} [{:<3.1f} s]    dev=({})'.format(
                    epoch + 1, loss, training_time, utils.format_metric(dev_result))

                # Test
                if self.test_epoch > 0 and epoch % self.test_epoch  == 0:
                    test_result = self.evaluate(data_dict['test'], self.topk, self.metrics)
                    logging_str += ' test=({})'.format(utils.format_metric(test_result))
                    self.mlflow_log_metrics(self.head_to_dict(dev_result,'test'),epoch)
                testing_time = self._check_time()
                logging_str += ' [{:<.1f} s]'.format(testing_time)

                # Save model and early stop
                if max(main_metric_results) == main_metric_results[-1] or \
                        (hasattr(model, 'stage') and model.stage == 1):
                    model.save_model()
                    logging_str += ' *'
                logging.info(logging_str)

                if self.early_stop > 0 and self.eval_termination(main_metric_results):
                    logging.info("Early stop at %d based on dev result." % (epoch + 1))
                    break
        except KeyboardInterrupt:
            logging.info("Early stop manually")
            exit_here = input("Exit completely without evaluation? (y/n) (default n):")
            if exit_here.lower().startswith('y'):
                logging.info(os.linesep + '-' * 45 + ' END: ' + utils.get_time() + ' ' + '-' * 45)
                exit(1)

        # Find the best dev result across iterations
        best_epoch = main_metric_results.index(max(main_metric_results))
        logging.info(os.linesep + "Best Iter(dev)={:>5}\t dev=({}) [{:<.1f} s] ".format(
            best_epoch + 1, utils.format_metric(dev_results[best_epoch]), self.time[1] - self.time[0]))
        model.load_model()
    
    def evaluate(self, dataset, topks: list, metrics: list):
        """
        Evaluate the results for an input dataset.
        :return: result dict (key: metric@k)
        """
        predictions = self.predict(dataset)
        return self.evaluate_method(predictions, topks, metrics)

    def predict(self, dataset) -> np.ndarray:
        """
        The returned prediction is a 2D-array, each row corresponds to all the candidates,
        and the ground-truth item poses the first.
        Example: ground-truth items: [1, 2], 2 negative items for each instance: [[3,4], [5,6]]
                 predictions like: [[1,3,4], [2,5,6]]
        """
        dataset.model.eval()
        predictions = list()
        dl = DataLoader(dataset, batch_size=self.eval_batch_size, shuffle=False, num_workers=self.num_workers,
                        collate_fn=dataset.collate_batch, pin_memory=self.pin_memory)
        for batch in tqdm(dl, leave=False, ncols=100, mininterval=1, desc='Predict'):
            import pdb;pdb.set_trace()
            prediction = dataset.model(utils.batch_to_gpu(batch, dataset.model.device))['prediction']
            predictions.extend(prediction.cpu().data.numpy())
        predictions = np.array(predictions)

        if dataset.model.test_all:
            rows, cols = list(), list()
            for i, u in enumerate(dataset.data['user_id']):
                clicked_items = list(dataset.corpus.train_clicked_set[u] | dataset.corpus.residual_clicked_set[u])
                idx = list(np.ones_like(clicked_items) * i)
                rows.extend(idx)
                cols.extend(clicked_items)
            predictions[rows, cols] = -np.inf

        return predictions

    def print_res(self, dataset) -> str:
        """
        Construct the final result string before/after training
        :return: test result string
        """
        result_dict = self.evaluate(dataset, self.topk, self.metrics)
        #res_str = '(' + utils.format_metric(result_dict) + ')'
        return result_dict
    
    def fit(self, dataset, epoch=-1) -> float:
        model = dataset.model
        if model.optimizer is None:
            model.optimizer = self._build_optimizer(model)
        dataset.actions_before_epoch()  # must sample before multi thread start

        model.train()
        loss_lst = list()
        dl = DataLoader(dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers,
                        collate_fn=dataset.collate_batch, pin_memory=self.pin_memory)
        for batch in tqdm(dl, leave=False, desc='Epoch {:<3}'.format(epoch), ncols=100, mininterval=1):
            batch = utils.batch_to_gpu(batch, model.device)
            model.optimizer.zero_grad()
            out_dict = model(batch)
            loss = model.loss(out_dict)
            loss.backward()
            model.optimizer.step()
            loss_lst.append(loss.detach().cpu().data.numpy())
        return np.mean(loss_lst).item()

In [ ]:
runner = BaseRunnerv1(args)

## Interaction

In [ ]:
## list of all items present
item_id = [list(range(1,3707))]
## interaction history of items
history_items = [[907, 2915,  335, 1248,  514,  853, 3204,  312, 1157, 1161, 1486, 2522,
         528, 2006, 1169, 1176, 1193, 3108,  882, 1092]]
## user id
user_id = [6034]
## lengths
lengths = [len(x) for x in history_items]
batch_size = len(user_id)
sample = {'user_id':torch.tensor(user_id),'item_id':torch.tensor(item_id),'history_items':torch.tensor(history_items),\
         'lengths':torch.tensor(lengths),'batch_size':batch_size}
# dl = DataLoader(data_dict['test'], batch_size=256, shuffle=False, num_workers=5,
#                         collate_fn=data_dict['test'].collate_batch, pin_memory=0)

In [ ]:
model_check(sample)

In [ ]:
pred = model_check(sample)['prediction']
evaluations = dict()
sort_idx = (-pred).argsort(axis=1)
#gt_rank = np.argwhere(sort_idx == 0)[:, 1] + 1

In [ ]:
batch

In [ ]:
for batch in dl:
    import pdb;pdb.set_trace()
    model_check(batch)

In [ ]:
dl = DataLoader(data_dict['train'], batch_size=256, shuffle=False, num_workers=5,
                        collate_fn=data_dict['train'].collate_batch, pin_memory=0)

In [ ]:
predictions = list()
dl = DataLoader(data_dict['test'], batch_size=256, shuffle=False, num_workers=5,
                        collate_fn=data_dict['test'].collate_batch, pin_memory=0)
for batch in tqdm(dl, leave=False, ncols=100, mininterval=1, desc='Predict'):
    prediction = data_dict['test'].model(utils.batch_to_gpu(batch, data_dict['test'].model.device))['prediction']
    predictions.extend(prediction.cpu().data.numpy())
predictions = np.array(predictions)

In [ ]:
predictions.shape

In [ ]:
evaluations = dict()
sort_idx = (-predictions).argsort(axis=1)
gt_rank = np.argwhere(sort_idx == 0)[:, 1] + 1

In [ ]:
sort_idx

In [ ]:
predictions.shape

In [ ]:
gt_rank

In [ ]:
model 

In [ ]:
if data_dict['test'].model.test_all:
    print('hello')

In [ ]:
neg_items = [1584, 3385, 2862, 3106,  533, 3579, 2214,  568, 1827, 2670, 1272, 2921,
        3337,   68, 2659,  725, 3585, 1629, 2742,  124, 1471,  395, 1667, 3166,
        3422, 1698, 1437, 2435, 2303,  534, 3410, 1083, 2391, 2970,  166,  374,
        1813,  582,  455, 2460, 3561, 2304,  413, 1816, 1600, 3132,  182, 1054,
        3523, 2507, 2819, 3415, 2545, 2871,  559, 2083, 1699,  229, 3633, 3626,
        1921, 3238,  513, 1958, 3006, 1513, 2524, 1229, 2233, 1348, 3497, 1352,
        2399, 3003, 1157, 3452, 3347, 1824, 3152,  707,   88,  105,  241, 1617,
        1346, 3645, 3561, 3022, 2208,  804, 1445, 2593,  596, 3349, 2211,  931,
         165, 3655,  886, 3155]
user_id = 6034
pos_items = [ 419, 1856, 1328, 2746, 1605, 1345,  662,  365, 1862,  424, 2516,  941,
        1661, 2105,  316,  407,  416, 1565, 1972,  815]

In [ ]:
dl = DataLoader(data_dict['dev'], batch_size=256, shuffle=False, num_workers=5,
                        collate_fn=data_dict['dev'].collate_batch, pin_memory=0)

In [ ]:

prediction = dataset.model(utils.batch_to_gpu(batch, dataset.model.device))['prediction']
            predictions.extend(prediction.cpu().data.numpy())

In [ ]:
for batch in dl:
    model_check(batch)